In [13]:
import pandas as pd
import numpy as np
import re
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Embedding
from keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [14]:
# 1. Chargement des données
train_data = pd.read_csv('SQLIV3_cleaned2.csv')
test_data = pd.read_csv('sqliv2_utf8.csv')


In [19]:
print(train_data)

                                                Sentence  Label
0                      " or pg_sleep  (  __TIME__  )  --      1
1      create user name identified by pass123 tempora...      1
2       AND 1  =  utl_inaddr.get_host_address   (    ...      1
3       select * from users where id  =  '1' or @ @1 ...      1
4       select * from users where id  =  1 or 1#"  ( ...      1
...                                                  ...    ...
30609              DELETE FROM door WHERE grow = 'small'      0
30610                               DELETE FROM tomorrow      0
30611                       SELECT wide ( s )  FROM west      0
30612       SELECT * FROM  ( SELECT slide FROM breath )       0
30613                           SELECT TOP 3 * FROM race      0

[30600 rows x 2 columns]


In [16]:
train_data.duplicated().sum()

12

In [17]:
# 2. Suppression des doublons (en gardant la première occurrence)
train_data.drop_duplicates(subset='Sentence', keep='first', inplace=True)
test_data.drop_duplicates(subset='Sentence', keep='first', inplace=True)



In [18]:
train_data.duplicated().sum()

0

In [7]:
# 3. Affichage de la structure des datasets
print("Structure du train_data :")
print(train_data.info())
print("\nStatistiques :")
print(train_data.describe(include='all'))

print("\nStructure du test_data :")
print(test_data.info())

Structure du train_data :
<class 'pandas.core.frame.DataFrame'>
Index: 30600 entries, 0 to 30613
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  30600 non-null  object
 1   Label     30600 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 717.2+ KB
None

Statistiques :
                                 Sentence         Label
count                               30600  30600.000000
unique                              30600           NaN
top     " or pg_sleep  (  __TIME__  )  --           NaN
freq                                    1           NaN
mean                                  NaN      0.370654
std                                   NaN      0.482988
min                                   NaN      0.000000
25%                                   NaN      0.000000
50%                                   NaN      0.000000
75%                                   NaN      1.000000
max                           

In [7]:
# 4. Nettoyage minimal (on garde les caractères spéciaux)
def clean_text(text):
    return str(text).strip()

train_data['Sentence'] = train_data['Sentence'].apply(clean_text)
test_data['Sentence'] = test_data['Sentence'].apply(clean_text)

In [8]:
train_data

,Sentence,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,AND 1 = utl_inaddr.get_host_address ( (...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
...,...,...
30609,DELETE FROM door WHERE grow = 'small',0
30610,DELETE FROM tomorrow,0
30611,SELECT wide ( s ) FROM west,0
30612,SELECT * FROM ( SELECT slide FROM breath ),0


In [9]:
# 5. Tokenisation
vocab_size = 15000
tokenizer = Tokenizer(
    num_words=vocab_size,
    oov_token="<OOV>",
    filters='',        # Garde les caractères spéciaux
    lower=False
)
tokenizer.fit_on_texts(train_data['Sentence'])


In [20]:
print(train_data)

                                                Sentence  Label
0                      " or pg_sleep  (  __TIME__  )  --      1
1      create user name identified by pass123 tempora...      1
2       AND 1  =  utl_inaddr.get_host_address   (    ...      1
3       select * from users where id  =  '1' or @ @1 ...      1
4       select * from users where id  =  1 or 1#"  ( ...      1
...                                                  ...    ...
30609              DELETE FROM door WHERE grow = 'small'      0
30610                               DELETE FROM tomorrow      0
30611                       SELECT wide ( s )  FROM west      0
30612       SELECT * FROM  ( SELECT slide FROM breath )       0
30613                           SELECT TOP 3 * FROM race      0

[30600 rows x 2 columns]


In [10]:
# 6. Séquences et padding
max_len = int(np.percentile([len(x.split()) for x in train_data['Sentence']], 95))
X = tokenizer.texts_to_sequences(train_data['Sentence'])
X = pad_sequences(X, padding='post', maxlen=max_len)
y = train_data['Label'].astype('int')


In [11]:
# 7. Split train/val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# 8. Architecture du modèle LSTM
embedding_dim = 256
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    LSTM(256, return_sequences=True),
    Dropout(0.3),
    LSTM(128),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

c:\Users\ASUS ROG\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
# 9. Compilation du modèle
optimizer = optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [14]:
# 11. Entraînement
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(X_val, y_val),
)

Epoch 1/30
383/383 ━━━━━━━━━━━━━━━━━━━━ 121s 243ms/step - accuracy: 0.8587 - loss: 0.2992 - val_accuracy: 0.9933 - val_loss: 0.0419
Epoch 2/30
383/383 ━━━━━━━━━━━━━━━━━━━━ 93s 242ms/step - accuracy: 0.9932 - loss: 0.0393 - val_accuracy: 0.9962 - val_loss: 0.0340
Epoch 3/30
383/383 ━━━━━━━━━━━━━━━━━━━━ 88s 231ms/step - accuracy: 0.9953 - loss: 0.0303 - val_accuracy: 0.9959 - val_loss: 0.0253
Epoch 4/30
383/383 ━━━━━━━━━━━━━━━━━━━━ 94s 245ms/step - accuracy: 0.9954 - loss: 0.0281 - val_accuracy: 0.9959 - val_loss: 0.0254
Epoch 5/30
383/383 ━━━━━━━━━━━━━━━━━━━━ 113s 296ms/step - accuracy: 0.9955 - loss: 0.0273 - val_accuracy: 0.9959 - val_loss: 0.0253
Epoch 6/30
383/383 ━━━━━━━━━━━━━━━━━━━━ 110s 287ms/step - accuracy: 0.9955 - loss: 0.0274 - val_accuracy: 0.9964 - val_loss: 0.0229
Epoch 7/30
383/383 ━━━━━━━━━━━━━━━━━━━━ 105s 273ms/step - accuracy: 0.9959 - loss: 0.0253 - val_accuracy: 0.9964 - val_loss: 0.0229
Epoch 8/30
383/383 ━━━━━━━━━━━━━━━━━━━━ 90s 235ms/step - accuracy: 0.9957 - los

In [15]:
# 12. Préparation des données de test
X_test = tokenizer.texts_to_sequences(test_data['Sentence'])
X_test = pad_sequences(X_test, padding='post', maxlen=max_len)
y_test = test_data['Label'].astype('int')


In [16]:
# 13. Évaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'\n✅ Test Accuracy: {test_accuracy:.4f}')

1054/1054 ━━━━━━━━━━━━━━━━━━━━ 45s 43ms/step - accuracy: 0.9577 - loss: 0.3111

✅ Test Accuracy: 0.9817


In [ ]:
from sklearn.metrics import classification_report

# 1. Récupérer uniquement les lignes spam (Label = 1)
sqli_only = test_data[test_data['Label'] == 1].copy()
print(f"\n📌 Nombre total des injections sql dans test_data : {len(sqli_only)}")


📌 Nombre total de requêtes spam dans test_data : 11424


In [25]:
# 2. Nettoyage si besoin
sqli_only['Sentence'] = sqli_only['Sentence'].apply(clean_text)


In [26]:
# 3. Tokenisation + Padding
X_sqli = tokenizer.texts_to_sequences(sqli_only['Sentence'])
X_sqli = pad_sequences(X_sqli, padding='post', maxlen=max_len)

In [27]:
# 4. Prédiction
sqli_preds = model.predict(X_sqli)
sqli_preds_labels = (sqli_preds > 0.5).astype(int)

357/357 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step


In [28]:
# 5. Calcul du nombre de spams correctement détectés
true_positives = np.sum(sqli_preds_labels == 1)
total_sqli = len(sqli_only)
detection_rate = (true_positives / total_sqli) * 100

In [ ]:
# 6. Affichage
print(f"✅ les injections correctement détectés : {true_positives}")
print(f"📊 Taux de détection : {detection_rate:.2f}%")


✅ Spams correctement détectés : 11368
📊 Taux de détection : 99.51%
